In [155]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import keras
import tensorflow as tf
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import GridSearchCV
import sklearn
from sklearn.model_selection import RandomizedSearchCV

from keras.wrappers.scikit_learn import KerasClassifier

from skopt import dummy_minimize


In [2]:
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

In [3]:
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

Train: X=(60000, 28, 28), y=(60000,)
Test: X=(10000, 28, 28), y=(10000,)


In [12]:
def load_dataset():
    (trainX, trainY), (testX, testY) = fashion_mnist.load_data()
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

In [13]:
# scale pixels
def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

In [105]:
def define_model(optimizer):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(units = 100 , activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [106]:
model = KerasClassifier(build_fn=define_model,epochs = 10, batch_size=32)

units=[50,100,150]
optimizer = ['Adam','SGD']
param_grid = {'units'=units,'optimizer':optimizer}

<ipython-input-106-31d695b36cfe>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=define_model,epochs = 10, batch_size=32)


In [107]:
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [108]:
grid_result = grid.fit(trainX, trainy)

Epoch 1/10
1500/1500 [==============================] - 24s 15ms/step - loss: 5.2380 - accuracy: 0.5977
Epoch 2/10
1500/1500 [==============================] - 29s 19ms/step - loss: 0.9824 - accuracy: 0.6106
Epoch 3/10
1500/1500 [==============================] - 38s 25ms/step - loss: 0.9756 - accuracy: 0.5966
Epoch 4/10
1500/1500 [==============================] - 31s 21ms/step - loss: 0.8865 - accuracy: 0.6298
Epoch 5/10
1500/1500 [==============================] - 31s 21ms/step - loss: 0.8234 - accuracy: 0.6428
Epoch 6/10
1500/1500 [==============================] - 31s 20ms/step - loss: 0.7697 - accuracy: 0.6715
Epoch 7/10
1500/1500 [==============================] - 30s 20ms/step - loss: 0.6678 - accuracy: 0.7248
Epoch 8/10
1500/1500 [==============================] - 29s 19ms/step - loss: 0.5664 - accuracy: 0.7624
Epoch 9/10
1500/1500 [==============================] - 23s 16ms/step - loss: 0.4731 - accuracy: 0.8168
Epoch 10/10
375/375 [==============================] - 3s 7ms/st

In [26]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [109]:
print(grid.best_score_,grid.best_params_)

0.8479333400726319 {'optimizer': 'Adam'}


In [110]:
def define_model(optimizer,units):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(units = units , activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [114]:
model = KerasClassifier(build_fn=define_model,epochs = 5, batch_size=32)

units=[50,100,150]
optimizer = ['Adam','SGD']
param_grid = {'units':units,'optimizer':optimizer}

<ipython-input-114-96c6212e2dc4>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=define_model,epochs = 5, batch_size=32)


In [115]:
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [116]:
grid_result = grid.fit(trainX, trainy)

Epoch 1/5
1500/1500 [==============================] - 22s 14ms/step - loss: 4.2548 - accuracy: 0.7029
Epoch 2/5
1500/1500 [==============================] - 24s 16ms/step - loss: 0.7148 - accuracy: 0.7568
Epoch 3/5
1500/1500 [==============================] - 25s 17ms/step - loss: 0.6182 - accuracy: 0.7820
Epoch 4/5
1500/1500 [==============================] - 25s 17ms/step - loss: 0.5645 - accuracy: 0.8003
Epoch 5/5
375/375 [==============================] - 3s 8ms/step - loss: 0.5066 - accuracy: 0.8222
Epoch 1/5
1500/1500 [==============================] - 33s 22ms/step - loss: 4.5501 - accuracy: 0.4657
Epoch 2/5
1500/1500 [==============================] - 32s 21ms/step - loss: 1.3833 - accuracy: 0.4367
Epoch 3/5
1500/1500 [==============================] - 26s 18ms/step - loss: 1.3054 - accuracy: 0.4536
Epoch 4/5
1500/1500 [==============================] - 25s 17ms/step - loss: 1.2238 - accuracy: 0.4771
Epoch 5/5
375/375 [==============================] - 3s 7ms/step - loss: 1.02

In [117]:
print(grid.best_score_,grid.best_params_)

0.7218166589736938 {'optimizer': 'Adam', 'units': 50}


In [118]:
means = grid.cv_results_['mean_test_score']
parameters = grid.cv_results_['params']
for mean,parameter in zip(means,parameters):
    print (mean,parameter)

0.7218166589736938 {'optimizer': 'Adam', 'units': 50}
0.5274666666984558 {'optimizer': 'Adam', 'units': 100}
0.6507333278656006 {'optimizer': 'Adam', 'units': 150}
0.6306666672229767 {'optimizer': 'SGD', 'units': 50}
0.646699994802475 {'optimizer': 'SGD', 'units': 100}
0.4827500030398369 {'optimizer': 'SGD', 'units': 150}


In [119]:
#---------------------------------------RANDOM SEARCH -------------------------------------------------------------->

In [145]:
def define_model(units,learning_rate):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(units = units , activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    optimizer = SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [146]:
from scipy import stats

In [147]:
model = KerasClassifier(build_fn=define_model,epochs = 5, batch_size=32)


learning_rate = stats.uniform(0,0.1)
units=stats.randint(50,100)
param_dist = dict(units=units,learning_rate=learning_rate)

<ipython-input-147-e448b3554be6>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=define_model,epochs = 5, batch_size=32)


In [148]:
random_search = RandomizedSearchCV(estimator=model, 
                                   param_distributions=param_dist,
                                   n_iter=1,
                                   n_jobs=4) 

In [149]:
random_search.fit(trainX, trainy)

Epoch 1/5
1875/1875 [==============================] - 35s 19ms/step - loss: 26.2576 - accuracy: 0.5249
Epoch 2/5
1875/1875 [==============================] - 32s 17ms/step - loss: 0.8202 - accuracy: 0.6857
Epoch 3/5
1875/1875 [==============================] - 33s 18ms/step - loss: 0.7332 - accuracy: 0.7322
Epoch 4/5
1875/1875 [==============================] - 33s 18ms/step - loss: 0.6958 - accuracy: 0.7483
Epoch 5/5
1875/1875 [==============================] - 34s 18ms/step - loss: 0.6632 - accuracy: 0.7614


RandomizedSearchCV(estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001FC0330F250>,
                   n_iter=1, n_jobs=4,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FC0330F6A0>,
                                        'units': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FC699CB790>})

In [152]:
print(random_search.best_score_,random_search.best_params_)

0.34811666160821914 {'learning_rate': 0.024051385923440194, 'units': 50}


In [154]:
pip install scikit-optimize

Note: you may need to restart the kernel to use updated packages.
